# Classes definition

## class SystemData:

In [ ]:
from tabulate import tabulate
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import statistics
from statistics import pvariance
import pprint

class SystemData:

  HOURS_IN_YEAR = 8760

  def __init__(self, reliabilityData, loadData, fromToData, protectorData, switchData, alternativeFeederData):
    self._reliabilityData = reliabilityData
    self._loadData = loadData
    self._fromToData = fromToData
    self._protectorData = protectorData
    self._switchData = switchData
    self._alternativeFeederData = alternativeFeederData
    self._calculateAdjacencyMatrix()
    self._calculateReachMatrix()

  @property
  def switchData(self):
    return self._switchData

  @property
  def alternativeFeederData(self):
    return self._alternativeFeederData

  @property
  def reliabilityData(self):
    return self._reliabilityData

  @property
  def loadData(self):
    return self._loadData

  @property
  def fromToData(self):
    return self._fromToData

  @property
  def adjacencyMatrix(self):
    return self._A.copy()

  @property
  def reachMatrix(self):
      return self._R.copy()

  @property
  def protectorData(self):
      return self._protectorData

  def getNumberComponents(self):
    return len(self._reliabilityData)

  def getNodeLoad(self, indexLoad):
    return self._loadData[indexLoad][1]

  def getConsumersLoad(self, indexLoad):
    return self._loadData[indexLoad][2]

  def getCargaLoad(self, indexLoad):
    return self._loadData[indexLoad][3]

  def getNumberOfLoads(self):
    return len(self._loadData )

  def getFailureRateHours(self, indexComponent):
    return self._reliabilityData[indexComponent][2]/SystemData.HOURS_IN_YEAR

  def getMeanTimeRepairHours(self, indexComponent):
    return self._reliabilityData[indexComponent][3]

  def getProtector(self, indexComponent):
    return self._protectorData[indexComponent][1]

  def getSwitch(self, indexComponent):
    return self._switchData[indexComponent][1]

  def getAlternativeFeeder(self, indexComponent):
    return self._alternativeFeederData[indexComponent][1]

  def _calculateAdjacencyMatrix(self):
    n_nodes = np.max(self._fromToData)
    n_lines = np.shape(self._fromToData)[0]
    A = np.zeros((n_nodes, n_nodes))
    for line in range( n_lines ):
      node_from   = self._fromToData[line][0] - 1
      node_to     = self._fromToData[line][1] - 1
      A[node_from][node_to] = 1
    self._A = A

  def _calculateReachMatrix(self):
    n_nodes = np.max(fromToData)
    # Range Matrix
    A = self._A.astype(int)
    I = np.eye(n_nodes)
    R_aux = np.linalg.matrix_power(A + I, n_nodes - 1)
    R = (R_aux >= 1).astype(int)
    self._R = R

  def showReliabilitySystemData(self, title):
    tab_text = [["Component", "Length (km)", "Failure rate 𝝀 (failure/year)", "Mean Time to repair r (h)"]]
    for k in range( np.shape(self._reliabilityData)[0] ):
      tab_text.append(
          [(self._reliabilityData[k][0]),
          "{:.2f}".format(self._reliabilityData[k][1]),
          "{:.2f}".format(self._reliabilityData[k][2]),
          "{:.2f}".format(self._reliabilityData[k][3])
          ] )
    print(title)
    print(tabulate(tab_text))

  def showLoadSystemData(self, title):
    tab_text = [["Name", "Circuit node", "Consumers", "Load (kW)"]]
    for k in range( np.shape(self._loadData)[0] ):
      tab_text.append(
          [(self._loadData[k][0]),
          "{:.0f}".format(self._loadData[k][1]),
          "{:.0f}".format(self._loadData[k][2]),
          "{:.0f}".format(self._loadData[k][3])
          ] )
    print(title)
    print(tabulate(tab_text))


####################################################################

####################################################################

def getIndexes(output, isPrint=False):

  SAIFI = output['Num_int_cons'] / output['NumConsumersTotal']
  SAIDI = output['NumHoursInterrup'] / output['NumConsumersTotal']
  if SAIFI == 0:
    CAIDI = 0
  else:
    CAIDI = SAIDI / SAIFI
  #if output['Num_int_cons'] > 0:
  # CAIDI = (output['NumHoursInterrup'] / output['NumConsumersTotal']) / (output['Num_int_cons'] / output['NumConsumersTotal'])
  #else:
  # CAIDI = 0
  ASAI = 1 - (SAIDI / 8760)
  ASUI = 1 - ASAI
  #ASAI = 1 - (output['NumHoursInterrup'] / output['NumConsumersTotal'] / 8760)
  #ASUI = 1 - (1 - ((output['NumHoursInterrup'] / output['NumConsumersTotal'] / 8760)))
  ENS = output['Eint_cons']
  AENS = output['Eint_cons'] / output['NumConsumersTotal']

  # Dicionário de índices
  indexes = {
        'SAIFI': SAIFI,
        'SAIDI': SAIDI,
        'CAIDI': CAIDI,
        'ASAI': ASAI,
        'ASUI': ASUI,
        'ENS': ENS,
        'AENS': AENS
    }


  # Se a flag isPrint for verdadeira, imprimir os resultados
  if isPrint:

      print(f'SAIFI = {SAIFI}')
      print(f'SAIDI = {SAIDI}')
      print(f'CAIDI = {CAIDI}')
      print(f'ASAI = {ASAI}')
      print(f'ASUI = {ASUI}')
      print(f'ENS = {ENS}')
      print(f'AENS = {AENS}')


  return indexes



## class MonteCarloSimulation:

In [ ]:
class MonteCarloSimulation:

  COMPONENT_UP_STATE = 1
  COMPONENT_DOWN_STATE = 0

  def __init__(self, systemData, totalSimulationTime):
    self._systemData = systemData
    self._totalSimulationTime = totalSimulationTime

  @property
  def systemData(self):
    return self._systemData

  @property
  def totalSimulationTime(self):
    return self._totalSimulationTime

  def printIndexes(self, output):
    tab_text = [["Índice", "valor"]]
    tab_text.append( ['SAIFI', "{:.2f}".format(output['indexes']['SAIFI'])] )
    tab_text.append( ['SAIDI', "{:.2f}".format(output['indexes']['SAIDI'])] )
    tab_text.append( ['CAIDI', "{:.2f}".format(output['indexes']['CAIDI'])] )
    tab_text.append( ['ASAI',  "{:.6f}".format(output['indexes']['ASAI'])] )
    tab_text.append( ['ASUI',  "{:.6f}".format(output['indexes']['ASUI'])] )
    tab_text.append( ['ENS',   "{:.0f}".format(output['indexes']['ENS'])] )
    tab_text.append( ['AENS',  "{:.2f}".format(output['indexes']['AENS'])] )
    print(tabulate(tab_text))

  def execute(self):
    # Passo 1: Iniciar todos os componentes no estado de sucesso
    simulationTime       = 0
    numberExperiments    = [0]
    numberComponents     = self._systemData.getNumberComponents()
    stateMatriz          = np.ones((numberComponents, 1))
    timeDurationMatrix   = np.zeros((numberComponents, 1))
    simulationTimeRecord = [0]
    stateMatrizRecord    = stateMatriz.copy()

    # Inicializando variáveis
    numberOfLoads = self._systemData.getNumberOfLoads()
    NintConsTotal = 0
    Num_int_cons  = 0
    NumHoursInterrup = 0
    Eint_cons = 0
    impacted_consumers = 0
    loadConsumersOnes = np.ones((numberOfLoads, 1))
    loadConsumers     = np.zeros((numberOfLoads, 1))
    loadCarga         = np.zeros((numberOfLoads, 1))

    for idxLoad in range( numberOfLoads ):
      loadConsumers[idxLoad] = self._systemData.getConsumersLoad( idxLoad )
      loadCarga[idxLoad] = self._systemData.getCargaLoad( idxLoad )
    #end for

    impacted_consumers = 0
    impacted_consumers_fault = 0
    shed_load = 0
    while simulationTime < self._totalSimulationTime:
      numberExperiments.append( numberExperiments[-1] + 1)

      # Passo 2: Gerar uma duração no estado de sucesso ou falha para cada um dos componentes
      for indexComponent in range(numberComponents):
        componentState = stateMatriz[indexComponent]
        failureRate    = self._systemData.getFailureRateHours(indexComponent)
        repairRate     = 1 / self._systemData.getMeanTimeRepairHours(indexComponent)
        stateDurationTime = self.generateStateDurationTime(componentState, failureRate, repairRate)
        timeDurationMatrix[indexComponent][0] = stateDurationTime
      #end for

      # Passo 3: Identificar componente com menor tempo
      minTime  = np.min(timeDurationMatrix)
      idxComp  = np.argmin(timeDurationMatrix)

      # Passo 4: Se o componente foi para estado de falha, avaliar o impacto nos consumidores
      previousState = stateMatriz[idxComp][-1]
      if previousState == MonteCarloSimulation.COMPONENT_UP_STATE:
        # Falha
        stateMatriz[idxComp][0] = MonteCarloSimulation.COMPONENT_DOWN_STATE
        # Nós: protetor, seccionadora e alternativa
        pLT = self._systemData.getProtector(idxComp)
        sLT = self._systemData.getSwitch(idxComp)
        aLT = self._systemData.getAlternativeFeeder(idxComp)
        # Varrendo os nós de carga (A, B, C e D)
        BDP = np.zeros((numberOfLoads, 1))
        BDS = np.zeros((numberOfLoads, 1))
        BDA = np.zeros((numberOfLoads, 1))
        for idxLoad in range( numberOfLoads ):
          nodeLoad = self._systemData.getNodeLoad( idxLoad )
          R = self.systemData.reachMatrix
          BDP[idxLoad][0] = R[pLT-1][nodeLoad-1]
          if sLT != 0: # implica em seccionadora
            BDS[idxLoad][0] = R[sLT-1][nodeLoad-1]
          else:
            BDS[idxLoad][0] = 1
          if aLT != 0: # implica em alternativa e seccionadora
            BDA[idxLoad][0] = R[aLT-1][nodeLoad-1]
          else:
            BDA[idxLoad][0] = 0
        #endFor
        BDP = BDP.astype(int)
        BDS = BDS.astype(int)
        BDA = BDA.astype(int)
        loadState = np.abs(BDP - BDS) | BDA

        #print(self._systemData._reliabilityData[idxComp][0],
        #      '/pLT =', pLT, '/BDP = ', BDP.T, '/loadState = ', loadState.T)

        impacted_consumers_fault = sum( BDP * loadConsumers)
        impacted_consumers = sum( (loadConsumersOnes - loadState )* loadConsumers)
        shed_load = sum((loadConsumersOnes - loadState )* loadCarga) # potência cortada
      else:
        # componente reparado
        stateMatriz[idxComp][0] = MonteCarloSimulation.COMPONENT_UP_STATE

        # Passo 5: incrementar as funçoes teste dos índices de confiabilidade
        NumHoursInterrup = NumHoursInterrup + minTime*impacted_consumers # D_int_cons
        Num_int_cons = Num_int_cons + impacted_consumers_fault
        Eint_cons        = Eint_cons    + minTime*shed_load
      #end if


      # Passo 6: Atualizar o tempo
      simulationTime = simulationTime + minTime
      simulationTimeRecord.append(simulationTime)
      stateMatrizRecord = np.append(stateMatrizRecord, stateMatriz, axis=1)
    #edWhile

    # Outputs
    output = {
        'simulationTimeRecord': simulationTimeRecord,
        'stateMatrizRecord': stateMatrizRecord,
        'Eint_cons': float(Eint_cons[0]) if isinstance(Eint_cons, np.ndarray) else float(Eint_cons),
        'NumHoursInterrup': float(NumHoursInterrup[0]) if isinstance(NumHoursInterrup, np.ndarray) else float(NumHoursInterrup),
        'NumConsumersTotal': float(sum(loadConsumers)[0]) if isinstance(sum(loadConsumers), np.ndarray) else float(sum(loadConsumers)),
        'Num_int_cons': float(Num_int_cons[0]) if isinstance(Num_int_cons, np.ndarray) else float(Num_int_cons)
    }

    indexes = MonteCarloSimulation.getIndexes(output)
    output['indexes'] = indexes

    return output

  def generateStateDurationTime(self, componentState, failureRate, repairRate):
    U = random.random()
    durationTime = 0
    if componentState == 1:
      durationTime = - (1/failureRate)*np.log( U );
    if componentState == 0:
      durationTime = - (1/repairRate)*np.log( U );
    return durationTime;

  def getIndexes(output):
    SAIFI = output['Num_int_cons'] / output['NumConsumersTotal']
    SAIDI = output['NumHoursInterrup'] / output['NumConsumersTotal']
    if SAIFI > 0:
      CAIDI = SAIDI / SAIFI
    else:
      CAIDI = 0
    ASAI = 1 - (SAIDI / 8760)
    ASUI = 1 - ASAI
    ENS = output['Eint_cons']
    AENS = output['Eint_cons'] / output['NumConsumersTotal']

    indexes = {
          'SAIFI': SAIFI,
          'SAIDI': SAIDI,
          'CAIDI': CAIDI,
          'ASAI': ASAI,
          'ASUI': ASUI,
          'ENS': ENS,
          'AENS': AENS
      }
    return indexes


## class batchMonteCarlo:

In [ ]:
class BatchMonteCarlo:

  def __init__(self, monteCarloSimulation, NumSimulations):
    self._monteCarloSimulation = monteCarloSimulation
    self._NumSimulations       = NumSimulations

  def execute(self, mainCase=0):
    SAIFI = []
    SAIDI = []
    CAIDI = []
    ASAI  = []
    ASUI  = []
    ENS   = []
    AENS  = []

    for k in range(self._NumSimulations):
      output = self._monteCarloSimulation.execute()
      indexes = output['indexes']

      SAIFI.append( indexes['SAIFI'] )
      SAIDI.append( indexes['SAIDI'] )
      CAIDI.append( indexes['CAIDI'] )
      ASAI.append( indexes['ASAI'] )
      ASUI.append( indexes['ASUI'] )
      ENS.append( indexes['ENS'] )
      AENS.append( indexes['AENS'] )

    #  E[G] é uma média do valor esperado do índice
    E_SAIFI = sum(SAIFI) / self._NumSimulations
    E_SAIDI = sum(SAIDI) / self._NumSimulations
    E_CAIDI = sum(CAIDI) / self._NumSimulations
    E_ASAI  = sum(ASAI)  / self._NumSimulations
    E_ASUI  = sum(ASUI)  / self._NumSimulations
    E_ENS   = sum(ENS)   / self._NumSimulations
    E_AENS  = sum(AENS)  / self._NumSimulations

    # Cálculo das variâncias usando pvariance (variância populacional)
    Var_SAIFI = pvariance(SAIFI)
    Var_SAIDI = pvariance(SAIDI)
    Var_CAIDI = pvariance(CAIDI)
    Var_ASAI  = pvariance(ASAI)
    Var_ASUI  = pvariance(ASUI)
    Var_ENS   = pvariance(ENS)
    Var_AENS  = pvariance(AENS)

    # Cálculo do erro relativo (beta)
    Beta_SAIFI = (np.sqrt(Var_SAIFI / self._NumSimulations ) / E_SAIFI)
    Beta_SAIDI = (np.sqrt(Var_SAIDI / self._NumSimulations ) / E_SAIDI)
    Beta_CAIDI = (np.sqrt(Var_CAIDI / self._NumSimulations ) / E_CAIDI)
    Beta_ASAI  = (np.sqrt(Var_ASAI / self._NumSimulations ) / E_ASAI  )
    Beta_ASUI  = (np.sqrt(Var_ASUI / self._NumSimulations ) / E_ASUI  )
    Beta_ENS   = (np.sqrt(Var_ENS  / self._NumSimulations ) / E_ENS   )
    Beta_AENS  = (np.sqrt(Var_AENS / self._NumSimulations )     )


    case1Ref = [2.20, 6.00, 2.73, 0.999315, 0.000685, 84000, 28,0 ]
    case2Ref = [1.15, 3.91, 3.39, 0.999554, 0.000446, 54800, 18.3 ]
    case3Ref = [1.15, 2.58, 2.23, 0.999706, 0.000294, 35200, 11.7 ]
    case4Ref = [1.15, 1.80, 1.56, 0.999795, 0.000205, 25100, 8.4  ]

    if mainCase == 1:
      analitico = case1Ref
    elif mainCase == 2:
      analitico = case2Ref
    elif mainCase == 3:
      analitico = case3Ref
    elif mainCase == 4:
      analitico = case4Ref
    else:
      analitico = np.array(case1Ref)*0

    tab_text = [["Índice", "Analítico", "E(G)", "Und", "Beta"]]
    tab_text.append( ['SAIFI', "{:.4f}".format(analitico[0]), "{:.4f}".format(E_SAIFI),  "int/ano", "{:.4f}".format(Beta_SAIFI) ] )
    tab_text.append( ['SAIDI', "{:.4f}".format(analitico[1]), "{:.4f}".format(E_SAIDI),  "h/ano",   "{:.4f}".format(Beta_SAIDI) ] )
    tab_text.append( ['CAIDI', "{:.4f}".format(analitico[2]), "{:.4f}".format(E_CAIDI),  "h/int",   "{:.4f}".format(Beta_CAIDI) ] )
    tab_text.append( ['ASAI',  "{:.4f}".format(analitico[3]), "{:.4f}".format(E_ASAI),   "%",       "{:.4f}".format(Beta_ASAI) ] )
    tab_text.append( ['ASUI',  "{:.6f}".format(analitico[4]), "{:.6f}".format(E_ASUI),   "%",       "{:.4f}".format(Beta_ASUI) ] )
    tab_text.append( ['ENS',   "{:.0f}".format(analitico[5]), "{:.0f}".format(E_ENS),    "MWh/ano",      "{:.4f}".format(Beta_ENS) ] )
    tab_text.append( ['AENS',  "{:.4f}".format(analitico[6]), "{:.4f}".format(E_AENS),   "MWh/cons.ano", "{:.4f}".format(Beta_AENS) ] )

    print(tabulate(tab_text))



# Application

## Dados

In [ ]:
# ["Component", "Length (km)", "Failure rate 𝝀 (failure/year)", "Mean Time to repair r (h)"]
reliabilityData = [
  ["Section 1", 2, 0.2, 4],
  ["Section 2", 1, 0.1, 4],
  ["Section 3", 3, 0.3, 4],
  ["Section 4", 2, 0.2, 4],
  ["Distrib. a", 1, 0.2, 2],
  ["Distrib. b", 3, 0.6, 2],
  ["Distrib. c", 2, 0.4, 2],
  ["Distrib. d", 1, 0.2, 2],
]

'''
reliabilityData3 = [
  ["Section 1", 2, 0.2, 0.5],
  ["Section 2", 1, 0.1, 0.5],
  ["Section 3", 3, 0.3, 0.5],
  ["Section 4", 2, 0.2, 0.5],
  ["Distrib. a", 1, 0.2, 0.5],
  ["Distrib. b", 3, 0.6, 0.5],
  ["Distrib. c", 2, 0.4, 0.5],
  ["Distrib. d", 1, 0.2, 0.5],
]
'''


# ["Name", "Circuit node", "Consumers", "Load (kW)"]
loadData = [
    ["A", 6, 1000, 5000],
    ["B", 7, 800, 4000],
    ["C", 8, 700, 3000],
    ["D", 9, 500, 2000],
]

# Montagem da matriz
fromToData = np.array([
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [2, 6],
    [3, 7],
    [4, 8],
    [5, 9]
])

protectorData1 = [
  ["Section 1",  1],
  ["Section 2",  1],
  ["Section 3",  1],
  ["Section 4",  1],
  ["Distrib. a", 1],
  ["Distrib. b", 1],
  ["Distrib. c", 1],
  ["Distrib. d", 1],
]

protectorData2 = [
  ["Section 1",  1],
  ["Section 2",  1],
  ["Section 3",  1],
  ["Section 4",  1],
  ["Distrib. a", 6],
  ["Distrib. b", 7],
  ["Distrib. c", 8],
  ["Distrib. d", 9],
]

protectorData3 = [
  ["Section 1",  1],
  ["Section 2",  1],
  ["Section 3",  1],
  ["Section 4",  1],
  ["Distrib. a", 6],
  ["Distrib. b", 7],
  ["Distrib. c", 8],
  ["Distrib. d", 9],
]

protectorData4 = [
  ["Section 1",  1],
  ["Section 2",  1],
  ["Section 3",  1],
  ["Section 4",  1],
  ["Distrib. a", 6],
  ["Distrib. b", 7],
  ["Distrib. c", 8],
  ["Distrib. d", 9],
]

switchData1 = [
  ["Section 1", 0],
  ["Section 2", 0],
  ["Section 3", 0],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

switchData2 = [
  ["Section 1", 0],
  ["Section 2", 0],
  ["Section 3", 0],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

switchData3 = [
  ["Section 1", 0],
  ["Section 2", 3],
  ["Section 3", 4],
  ["Section 4", 5],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

switchData4 = [
  ["Section 1", 2],
  ["Section 2", 3],
  ["Section 3", 4],
  ["Section 4", 5],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

alternativeFeederData1 = [
  ["Section 1", 0],
  ["Section 2", 0],
  ["Section 3", 0],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]
alternativeFeederData2 = [
  ["Section 1", 0],
  ["Section 2", 0],
  ["Section 3", 0],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]
alternativeFeederData3 = [
  ["Section 1", 0],
  ["Section 2", 0],
  ["Section 3", 0],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

alternativeFeederData4 = [
  ["Section 1", 3],
  ["Section 2", 4],
  ["Section 3", 5],
  ["Section 4", 0],
  ["Distrib. a", 0],
  ["Distrib. b", 0],
  ["Distrib. c", 0],
  ["Distrib. d", 0],
]

## Caso 1

In [ ]:
dataCase1        = SystemData(reliabilityData, loadData, fromToData, protectorData1, switchData1, alternativeFeederData1)
monteCarlo1      = MonteCarloSimulation(dataCase1, 8760)
#output1 = monteCarlo1.execute()
#monteCarlo1.printIndexes(output1)
batchMonteCarlo1 = BatchMonteCarlo(monteCarlo1, 2000)
print('Caso 1')
batchMonteCarlo1.execute(1)

Caso 1
------  ---------  --------  ------------  ------
Índice  Analítico  E(G)      Und           Beta
SAIFI   2.2000     2.1885    int/ano       0.0154
SAIDI   6.0000     5.8866    h/ano         0.0227
CAIDI   2.7300     2.3755    h/int         0.0210
ASAI    0.9993     0.9993    %             0.0000
ASUI    0.000685   0.000672  %             0.0227
ENS     84000      82413     MWh/ano       0.0227
AENS    28.0000    27.4710   MWh/cons.ano  0.6237
------  ---------  --------  ------------  ------


## Caso 2

In [ ]:
dataCase2        = SystemData(reliabilityData, loadData, fromToData, protectorData2, switchData2, alternativeFeederData2)
monteCarlo2      = MonteCarloSimulation(dataCase2, 8760)
#output2          = monteCarlo2.execute()
#monteCarlo2.printIndexes(output2)
batchMonteCarlo2 = BatchMonteCarlo(monteCarlo2, 2000)
print('Caso 2')
batchMonteCarlo2.execute(2)

Caso 2
------  ---------  --------  ------------  ------
Índice  Analítico  E(G)      Und           Beta
SAIFI   1.1500     1.1313    int/ano       0.0190
SAIDI   3.9100     3.9100    h/ano         0.0301
CAIDI   3.3900     2.6804    h/int         0.0232
ASAI    0.9996     0.9996    %             0.0000
ASUI    0.000446   0.000446  %             0.0301
ENS     54800      54830     MWh/ano       0.0301
AENS    18.3000    18.2766   MWh/cons.ano  0.5494
------  ---------  --------  ------------  ------


## Caso 3

In [ ]:
dataCase3        = SystemData(reliabilityData, loadData, fromToData, protectorData3, switchData3, alternativeFeederData3)
monteCarlo3      = MonteCarloSimulation(dataCase3, 8760)
#output3          = monteCarlo3.execute()
#monteCarlo3.printIndexes(output3)
batchMonteCarlo3 = BatchMonteCarlo(monteCarlo3, 2000)
print('Caso 3')
batchMonteCarlo3.execute(3)

Caso 3
------  ---------  --------  ------------  ------
Índice  Analítico  E(G)      Und           Beta
SAIFI   1.1500     1.1942    int/ano       0.0179
SAIDI   2.5800     2.4440    h/ano         0.0290
CAIDI   2.2300     1.8534    h/int         0.0232
ASAI    0.9997     0.9997    %             0.0000
ASUI    0.000294   0.000279  %             0.0290
ENS     35200      33161     MWh/ano       0.0293
AENS    11.7000    11.0538   MWh/cons.ano  0.3237
------  ---------  --------  ------------  ------


## Caso 4

In [ ]:
dataCase4        = SystemData(reliabilityData, loadData, fromToData, protectorData4, switchData4, alternativeFeederData4)
monteCarlo4      = MonteCarloSimulation(dataCase4, 8760)
#output4          = monteCarlo4.execute()
#monteCarlo4.printIndexes(output4)
batchMonteCarlo4 = BatchMonteCarlo(monteCarlo4, 2000)
print('Caso 4')
batchMonteCarlo4.execute(4)

Caso 4
------  ---------  --------  ------------  ------
Índice  Analítico  E(G)      Und           Beta
SAIFI   1.1500     1.1552    int/ano       0.0182
SAIDI   1.8000     1.4630    h/ano         0.0227
CAIDI   1.5600     1.2929    h/int         0.0214
ASAI    0.9998     0.9998    %             0.0000
ASUI    0.000205   0.000167  %             0.0227
ENS     25100      20298     MWh/ano       0.0230
AENS    8.4000     6.7659    MWh/cons.ano  0.1556
------  ---------  --------  ------------  ------
